In [1]:
import os
import numpy as np
import tensorflow as tf
from datetime import datetime
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping

In [2]:
# Set the hyperparameter
batch_size = 32
img_height = 48
img_width = 48

In [3]:
def DNC(func):
    return tf.autograph.experimental.do_not_convert(func)

In [4]:
# Dataset dir path
train_data_dir = "./dataset/train"
test_data_dir = "./dataset/test"

In [5]:
# Get train dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    validation_split=0.2,
    subset="training",
    seed=1,
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),
)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.


In [6]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_dir,
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

Found 7178 files belonging to 7 classes.


In [7]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [8]:
# Use tf.autograph.experimental.do_not_convert() to suppresses warnings
train_nor_dataset = train_dataset.map(DNC(
    lambda x, y: (normalization_layer(x), y)))
test_nor_dataset = test_dataset.map(DNC(
    lambda x, y: (normalization_layer(x), y)))

In [9]:
class_names = train_dataset.class_names

print(class_names)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [10]:
num_classes = 7

In [11]:
model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./225, input_shape=(img_height, img_width, 1)),
    
    layers.Conv2D(16, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    
    layers.Conv2D(32, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, 3, padding="same", activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(num_classes)
])

In [12]:
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [13]:
model.compile(optimizer=sgd,
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 48, 48, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0

In [15]:
epochs = 50

In [16]:
os.makedirs("models", exist_ok=True)
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [19]:
history = model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=[
#         DNC(ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=10, verbose=1, mode="auto", min_lr=1e-05)),
#         DNC(ModelCheckpoint("models/%s.h5" % (start_time), monitor="val_loss", save_best_only=True, mode="min", verbose=1)),
#         DNC(TensorBoard(log_dir="logs/%s" % (start_time))),
#         DNC(EarlyStopping(monitor="val_loss", patience=3))
    ],
    use_multiprocessing=True,
    workers=5,
    max_queue_size=5
)

Epoch 1/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4934 - accuracy: 0.8241
Epoch 2/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4900 - accuracy: 0.8231
Epoch 3/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4928 - accuracy: 0.8255
Epoch 4/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4938 - accuracy: 0.8259
Epoch 5/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4817 - accuracy: 0.82820s - loss: 0.4851 - ac
Epoch 6/50
718/718 [==============================] - 16s 22ms/step - loss: 0.5014 - accuracy: 0.8224
Epoch 7/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4778 - accuracy: 0.8303
Epoch 8/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4854 - accuracy: 0.8301
Epoch 9/50
718/718 [==============================] - 16s 22ms/step - loss: 0.4778 - accuracy: 0.8316
Epoch 10/50
718/718 [==============================] - 16s 2

In [20]:
model.evaluate(test_dataset)

225/225 [==============================] - 4s 19ms/step - loss: 2.1102 - accuracy: 0.5177


[2.110214948654175, 0.5176929235458374]